# Create System Metadata

An instance of System Metadata is created for arbitrary values to demonstrate programmatic construction of system metadata using the python libraries.

First gather the values to be used for the system metadata. Normally these would be obtained by examining a file.

In [8]:
import notebook_utils as nbu
from d1_common.types import dataoneTypes
import hashlib

def checksumOfObject( o ):
    '''Compute the SHA-256 checksum of an arbitrary thing.
    '''
    m = hashlib.sha256()
    if isinstance(o, bytes):
        m.update(o)
    elif isinstance(o, str):
        m.update( bytes(o, "utf-8") )
    else:
        m.update( bytes( o ) )
    return m.hexdigest()


# Create a dictionary of properties to use
source = {
    "identifier":"test_identifier_01",
    "seriesIdentifier":"test",
    "formatId": "eml://ecoinformatics.org/eml-2.0.1",
    "mediaType": "text/xml",
    "fileName":"metadata.xml",
    "size":17181,
    "checksum": checksumOfObject("arbitrary thing to create checksum"),
    "checksum_algorithm": "SHA256",
    "submitter":"CN=urn:node:cnStageUNM1,DC=dataone,DC=org",
    "rightsHolder":"CN=testRightsHolder,DC=dataone,DC=org",
    "obsoletes":"test_identifier_00",
    "obsoletedBy": None,
}


Given the dictionary of values, create the system metadata and print the resulting XML.

In [9]:
system_metadata = dataoneTypes.systemMetadata()

# Set with public read-only defaults
# enable public read by adding read permission for the "Public" user
access_policy = dataoneTypes.accessPolicy()
access_rule = dataoneTypes.accessRule()
access_rule.subject.append("Public")
access_rule.permission.append(dataoneTypes.Permission('read'))
access_policy.append(access_rule)
system_metadata.accessPolicy = access_policy

# Replication Policy
replication_policy = dataoneTypes.replicationPolicy()
replication_policy.replicationAllowed = True
replication_policy.numberReplicas = 1
system_metadata.replicationPolicy = replication_policy

# Obsolescence
system_metadata.obsoletes = source.get("obsoletes", None)
system_metadata.obsoletedBy = source.get("obsoletedBy", None)

# Add properties
system_metadata.identifier = source.get("identifier")
system_metadata.seriesIdentifier = source.get("seriesIdentifier")
system_metadata.formatId = source.get("formatId")
system_metadata.size = source.get("size")

checksum = dataoneTypes.checksum( source.get("checksum") )
checksum.algorithm = source.get("checksum_algorithm")
system_metadata.checksum = checksum

system_metadata.submitter = source.get("submitter")
system_metadata.rightsHolder = source.get("rightsHolder")

#Media type information
media_type = dataoneTypes.mediaType()
media_type.name = source.get("mediaType")
system_metadata.mediaType = media_type
system_metadata.fileName = source.get("fileName")

print(nbu.asXml(system_metadata))

<?xml version="1.0" ?>
<ns1:systemMetadata xmlns:ns1="http://ns.dataone.org/service/types/v2.0">
  <identifier>test_identifier_01</identifier>
  <formatId>eml://ecoinformatics.org/eml-2.0.1</formatId>
  <size>17181</size>
  <checksum algorithm="SHA256">2daf664721aa2139df9831d9ea7f8e95f6483992ee96489181cf7041363f9e7c</checksum>
  <submitter>CN=urn:node:cnStageUNM1,DC=dataone,DC=org</submitter>
  <rightsHolder>CN=testRightsHolder,DC=dataone,DC=org</rightsHolder>
  <accessPolicy>
    <allow>
      <subject>Public</subject>
      <permission>read</permission>
    </allow>
  </accessPolicy>
  <replicationPolicy numberReplicas="1" replicationAllowed="true"/>
  <obsoletes>test_identifier_00</obsoletes>
  <mediaType name="text/xml"/>
  <fileName>metadata.xml</fileName>
</ns1:systemMetadata>

